In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-10 17:27:21,172] A new study created in memory with name: no-name-1998bdfb-1010-47d2-8bc4-ed29d1ebfa19
[I 2025-09-10 17:27:22,325] Trial 0 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 128, 'max_depth': 4}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-09-10 17:27:23,174] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 106, 'max_depth': 6}. Best is trial 1 with value: 0.7653631284916201.
[I 2025-09-10 17:27:23,807] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 103, 'max_depth': 14}. Best is trial 2 with value: 0.7690875232774674.
[I 2025-09-10 17:27:25,299] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 149, 'max_depth': 10}. Best is trial 2 with value: 0.7690875232774674.
[I 2025-09-10 17:27:26,474] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 78, 'max_depth': 20}. Best is trial 2 with value: 0.7690875

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350094
Best hyperparameters: {'n_estimators': 73, 'max_depth': 17}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.76


# Samplers in optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-10 17:28:09,193] A new study created in memory with name: no-name-e0c782dd-779b-4275-8375-8659edffd32c
[I 2025-09-10 17:28:09,755] Trial 0 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 92, 'max_depth': 10}. Best is trial 0 with value: 0.7597765363128491.
[I 2025-09-10 17:28:10,414] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 111, 'max_depth': 7}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-10 17:28:10,801] Trial 2 finished with value: 0.7783985102420857 and parameters: {'n_estimators': 64, 'max_depth': 16}. Best is trial 2 with value: 0.7783985102420857.
[I 2025-09-10 17:28:12,223] Trial 3 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 177, 'max_depth': 14}. Best is trial 2 with value: 0.7783985102420857.
[I 2025-09-10 17:28:13,372] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 127, 'max_depth': 10}. Best is trial 2 with value: 0.7783985

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7783985102420857
Best hyperparameters: {'n_estimators': 64, 'max_depth': 16}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.75


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-09-10 17:28:57,595] A new study created in memory with name: no-name-d8a1b698-6e4b-4f0a-b26c-3f6c8d23b21b
[I 2025-09-10 17:28:58,150] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-10 17:28:59,044] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-10 17:28:59,367] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-10 17:28:59,968] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-10 17:29:00,581] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


# Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

# Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-10 17:29:20,853] A new study created in memory with name: no-name-2be5a9e8-b52f-426a-973b-e8e83ef5c0a6
[I 2025-09-10 17:29:23,449] Trial 0 finished with value: 0.74487895716946 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 133, 'learning_rate': 0.23925092958002045, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.74487895716946.
[I 2025-09-10 17:29:23,813] Trial 1 finished with value: 0.7579143389199255 and parameters: {'classifier': 'RandomForest', 'n_estimators': 55, 'max_depth': 14, 'min_samples_split': 8, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 1 with value: 0.7579143389199255.
[I 2025-09-10 17:29:27,112] Trial 2 finished with value: 0.7188081936685289 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 291, 'learning_rate': 0.26170084621213496, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.7579143389199255.
[I 2025-09-10 17:

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1172606154372545, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.744879,2025-09-10 17:29:20.863686,2025-09-10 17:29:23.449327,0 days 00:00:02.585641,NaN,NaN,GradientBoosting,NaN,NaN,0.239251,6.0,7.0,7.0,133.0,COMPLETE
1,1,0.757914,2025-09-10 17:29:23.450694,2025-09-10 17:29:23.813620,0 days 00:00:00.362926,NaN,False,RandomForest,NaN,NaN,NaN,14.0,1.0,8.0,55.0,COMPLETE
2,2,0.718808,2025-09-10 17:29:23.817939,2025-09-10 17:29:27.111987,0 days 00:00:03.294048,NaN,NaN,GradientBoosting,NaN,NaN,0.261701,20.0,1.0,6.0,291.0,COMPLETE
3,3,0.783985,2025-09-10 17:29:27.112941,2025-09-10 17:29:27.684756,0 days 00:00:00.571815,NaN,False,RandomForest,NaN,NaN,NaN,20.0,10.0,5.0,110.0,COMPLETE
4,4,0.748603,2025-09-10 17:29:27.685873,2025-09-10 17:29:30.810326,0 days 00:00:03.124453,NaN,NaN,GradientBoosting,NaN,NaN,0.038289,19.0,10.0,8.0,88.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2025-09-10 17:30:16.368550,2025-09-10 17:30:16.399132,0 days 00:00:00.030582,0.186588,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.772812,2025-09-10 17:30:16.400116,2025-09-10 17:30:16.443351,0 days 00:00:00.043235,0.120022,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.748603,2025-09-10 17:30:16.444224,2025-09-10 17:30:16.481544,0 days 00:00:00.037320,0.148603,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.787709,2025-09-10 17:30:16.482520,2025-09-10 17:30:16.518266,0 days 00:00:00.035746,0.100410,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
GradientBoosting,11
RandomForest,11


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.747588
RandomForest,0.768241
SVM,0.775223


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [34]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np
from xgboost.callback import EarlyStopping

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Use sklearn API for XGBClassifier
    model = xgb.XGBClassifier(**param, use_label_encoder=False)

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-mlogloss")

    from xgboost.callback import EarlyStopping

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)],
        callbacks=[EarlyStopping(rounds=30), pruning_callback],
        verbose=False
      )



    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")

[I 2025-09-10 17:30:29,476] A new study created in memory with name: no-name-901b641d-b117-46d1-a718-96edea0c5ad3
[W 2025-09-10 17:30:29,480] Trial 0 failed with parameters: {'lambda': 0.6919380219319632, 'alpha': 0.00020976326296004777, 'eta': 0.1573671779797636, 'gamma': 3.213932307637444e-07, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.40251999329417576, 'colsample_bytree': 0.9815661269619125} because of the following error: TypeError("XGBClassifier.fit() got an unexpected keyword argument 'callbacks'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-911839262.py", line 42, in objective
    model.fit(
  File "/usr/local/lib/python3.12/dist-packages/xgboost/core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
TypeError: XGBClassifier.fit() got an unexpected keyword

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'callbacks'

In [35]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()

[W 2025-09-10 17:30:33,498] Study instance does not contain trials.


In [ ]:
import xgboost
print(xgboost.__version__)